In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
population_df = pd.read_csv('data/population.csv')
roads_network_df = pd.read_csv('data/roads_network.csv')
sample_submission_df = pd.read_csv('data/sample_submission.csv')

given_train_df = pd.read_csv('data/train.csv')

In [75]:
test_df = pd.read_csv('data/test.csv')

In [96]:
train_df = given_train_df.copy()

In [97]:
# test_df['postcode_short'] = train_df.apply(lambda x : x['postcode'][:x['postcode'].find(' ') +2], axis = 1)

def postcode_short_apply(x):
    if x.find(' ') != -1:
        return x[:x.find(' ') +2]
    else:
        return x

def postcode_short_short_apply(x):
    if x.find(' ') != -1:
        return x[:x.find(' ')]
    else:
        return x

train_df['postcode_short'] = train_df['postcode'].apply(postcode_short_apply)
train_df['postcode_short_1'] = train_df['postcode'].apply(postcode_short_short_apply)

In [78]:
test_df['postcode_short'] = test_df['postcode'].apply(postcode_short_apply)
test_df['postcode_short_1'] = test_df['postcode'].apply(postcode_short_short_apply)

In [98]:
train_df.shape, test_df.shape

((478741, 29), (121259, 64))

### Population IDVs

In [8]:
population_df = pd.read_feather('data_feather/population_df_cleaned.feather')

In [22]:
population_df.head()

,postcode,Rural Urban,All usual residents,Males,Females,Lives in a household,Lives in a communal establishment,Schoolchild or full-time student aged 4 and over at their non term-time address,Area (Hectares),Density (number of persons per hectare),postcode_cleaned,postcode_short_1
0,AL1 1,Total,5453,2715,2738,5408,45,75,225.63,24.2,AL1 1,AL1
1,AL1 2,Total,6523,3183,3340,6418,105,77,286.59,22.8,AL1 2,AL1
2,AL1 3,Total,4179,2121,2058,4100,79,46,97.12,43.0,AL1 3,AL1
3,AL1 4,Total,9799,4845,4954,9765,34,285,244.75,40.0,AL1 4,AL1
4,AL1 5,Total,10226,5129,5097,10211,15,133,200.93,50.9,AL1 5,AL1


In [99]:
train_df = train_df.merge(population_df[[col for col in population_df.columns if 'postcode' not in col ] + ['postcode_cleaned']], how = 'left', left_on='postcode_short', right_on='postcode_cleaned')
train_df.shape

(478741, 39)

In [81]:
test_df = test_df.merge(population_df[[col for col in population_df.columns if 'postcode' not in col ] + ['postcode_cleaned']], how = 'left', left_on='postcode_short', right_on='postcode_cleaned')
test_df.shape

(121259, 39)

In [26]:
train_df.isna().sum()

Accident_ID                                                                            0
Police_Force                                                                           0
Number_of_Vehicles                                                                     0
Number_of_Casualties                                                                   0
Date                                                                                   0
Day_of_Week                                                                            0
Time                                                                                  14
Local_Authority_(District)                                                             0
Local_Authority_(Highway)                                                              0
1st_Road_Class                                                                         0
1st_Road_Number                                                                        0
Road_Type            

In [34]:
test_df.head()

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,...,Rural Urban,All usual residents,Males,Females,Lives in a household,Lives in a communal establishment,Schoolchild or full-time student aged 4 and over at their non term-time address,Area (Hectares),Density (number of persons per hectare),postcode_cleaned
0,1,34,2,1,19/12/12,7,13:20,344,E10000032,4,...,Total,8929.0,4268.0,4661.0,8855.0,74.0,142.0,2575.85,3.5,OX3 9
1,2,5,2,1,02/11/12,4,7:53,102,E09000026,3,...,Total,6360.0,3038.0,3322.0,6341.0,19.0,43.0,265.07,24.0,S35 4
2,3,1,2,1,02/11/12,4,16:00,531,E10000016,6,...,Total,9050.0,4301.0,4749.0,8492.0,558.0,115.0,327.07,27.7,BN21 2
3,4,1,1,1,06/05/12,1,16:50,7,E08000035,6,...,Total,2690.0,1326.0,1364.0,2685.0,5.0,43.0,6483.99,0.4,TA20 3
4,5,46,1,1,30/06/12,3,13:25,519,E10000031,3,...,Total,6875.0,3338.0,3537.0,6754.0,121.0,82.0,8404.23,0.8,DN20 0


In [45]:
import datetime
def time_categorize(x):
    try:
        if np.isnan(float(x)):
            return np.nan
    except:
        pass
    x = datetime.datetime.strptime(x,'%H:%M').time()
    if datetime.time(0,0,0) <= x <= datetime.time(8,0,0):
        return '0_8'
    if datetime.time(8,0,1) <= x <= datetime.time(16,0,0):
        return '8_16'
    if datetime.time(16,0,1) <= x <= datetime.time(23,59,59):
        return '16_24'
    


### RoadNetwork IDVs

In [20]:
roads_network_df_cleaned = pd.read_feather('data_feather/roads_network_df_cleaned.feather')
roads_network_df_cleaned = roads_network_df_cleaned.set_index('index')

In [35]:
roads_network_df_cleaned.head()

,roadClassi_,roadClassi_A Road,roadClassi_Motorway,roadFuncti_,roadFuncti_A Road,roadFuncti_Motorway,formOfWay_,formOfWay_Collapsed Dual Carriageway,formOfWay_Dual Carriageway,formOfWay_Roundabout,formOfWay_Single Carriageway,formOfWay_Slip Road,length_min,length_mean,length_max,distance to the nearest point on rd_min,distance to the nearest point on rd_mean,distance to the nearest point on rd_max,primaryRou_or,primaryRou_and
index,,,,,,,,,,,,,,,,,,,,
AB1,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.256769,1.256769,1.256769,1,1
AB1 9NN,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.834101,1.834101,1.834101,1,1
AB10 1UH,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.830243,1.830243,1.830243,1,1
AB10 1YL,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.835092,1.835092,1.835092,1,1
AB10 6AT,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.814373,1.814373,1.814373,1,1


In [100]:
t1 = train_df.merge(roads_network_df_cleaned, how = 'left', left_on='postcode_short_1', right_index=True)
t2 = t1.set_index('postcode_short')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2 = t2.set_index('postcode')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2.shape

(478741, 59)

In [101]:
train_df = t2.copy()
train_df.shape

(478741, 59)

In [40]:
train_df.isna().sum()

postcode                                                                               0
postcode_short                                                                         0
Accident_ID                                                                            0
Police_Force                                                                           0
Number_of_Vehicles                                                                     0
Number_of_Casualties                                                                   0
Date                                                                                   0
Day_of_Week                                                                            0
Time                                                                                  14
Local_Authority_(District)                                                             0
Local_Authority_(Highway)                                                              0
1st_Road_Class       

In [42]:
roads_network_df_cleaned.head()

,roadClassi_,roadClassi_A Road,roadClassi_Motorway,roadFuncti_,roadFuncti_A Road,roadFuncti_Motorway,formOfWay_,formOfWay_Collapsed Dual Carriageway,formOfWay_Dual Carriageway,formOfWay_Roundabout,formOfWay_Single Carriageway,formOfWay_Slip Road,length_min,length_mean,length_max,distance to the nearest point on rd_min,distance to the nearest point on rd_mean,distance to the nearest point on rd_max,primaryRou_or,primaryRou_and
index,,,,,,,,,,,,,,,,,,,,
AB1,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.256769,1.256769,1.256769,1,1
AB1 9NN,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.834101,1.834101,1.834101,1,1
AB10 1UH,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.830243,1.830243,1.830243,1,1
AB10 1YL,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.835092,1.835092,1.835092,1,1
AB10 6AT,0,1,0,0,1,0,0,0,0,0,1,0,2643.0,2643.0,2643.0,1.814373,1.814373,1.814373,1,1


In [84]:
t1 = test_df.merge(roads_network_df_cleaned, how = 'left', left_on='postcode_short_1', right_index=True)
t2 = t1.set_index('postcode_short')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2 = t2.set_index('postcode')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2.shape

(121259, 59)

In [85]:
test_df = t2.copy()
test_df.shape

(121259, 59)

In [102]:
train_df['Time_category']  = train_df['Time'].apply(time_categorize)
train_df.head()

,postcode,postcode_short,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),...,formOfWay_Slip Road,length_min,length_mean,length_max,distance to the nearest point on rd_min,distance to the nearest point on rd_mean,distance to the nearest point on rd_max,primaryRou_or,primaryRou_and,Time_category
0,OX3 9UP,OX3 9,1,34,2,1,19/12/12,7,13:20,344,...,0.0,29.0,29.0,29.0,0.014415,0.014415,0.014415,1.0,1.0,8_16
1,S35 4EZ,S35 4,2,5,2,1,02/11/12,4,7:53,102,...,0.0,38.0,38.0,38.0,0.022913,0.022913,0.022913,0.0,0.0,0_8
2,BN21 2XR,BN21 2,3,1,2,1,02/11/12,4,16:00,531,...,0.0,149.0,149.0,149.0,0.000574,0.000574,0.000574,0.0,0.0,8_16
3,TA20 3PT,TA20 3,4,1,1,1,06/05/12,1,16:50,7,...,0.0,359.0,359.0,359.0,0.055676,0.055676,0.055676,1.0,1.0,16_24
4,DN20 0QF,DN20 0,5,46,1,1,30/06/12,3,13:25,519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8_16


In [87]:
test_df['Time_category']  = test_df['Time'].apply(time_categorize)
test_df.head()

,postcode,postcode_short,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),...,formOfWay_Slip Road,length_min,length_mean,length_max,distance to the nearest point on rd_min,distance to the nearest point on rd_mean,distance to the nearest point on rd_max,primaryRou_or,primaryRou_and,Time_category
0,HX2 8WH,HX2 8,14,13,2,0,06/10/13,6,13:28,218,...,0.0,19.0,89.0,159.0,0.019355,0.019355,0.019355,1.0,1.0,8_16
1,RM8 1DD,RM8 1,17,13,2,0,22/04/13,7,9:30,157,...,0.0,940.0,940.0,940.0,0.014327,0.014327,0.014327,1.0,1.0,8_16
2,SE23 1NH,SE23 1,21,13,2,0,27/09/13,3,19:10,155,...,0.0,11.0,30.0,49.0,0.006889,0.006889,0.006889,1.0,1.0,16_24
3,HU10 7QS,HU10 7,23,13,2,0,13/03/13,4,9:19,26,...,0.0,637.0,637.0,637.0,0.031804,0.031804,0.031804,1.0,1.0,8_16
4,BD23 5JL,BD23 5,28,14,2,0,13/06/13,1,14:59,6,...,0.0,31.0,31.0,31.0,0.000447,0.000447,0.000447,1.0,1.0,8_16


In [103]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['year'] = train_df['Date'].dt.year
train_df['month'] = train_df['Date'].dt.month
train_df['hour'] = pd.to_datetime( train_df['Time']).dt.hour
train_df['minutes'] = pd.to_datetime( train_df['Time']).dt.minute

In [89]:
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df['year'] = test_df['Date'].dt.year
test_df['month'] = test_df['Date'].dt.month
test_df['hour'] = pd.to_datetime( test_df['Time']).dt.hour
test_df['minutes'] = pd.to_datetime( test_df['Time']).dt.minute

In [104]:
train_df.shape,test_df.shape

((478741, 64), (121259, 64))

In [105]:
cat_columns = ['Day_of_Week','1st_Road_Class', '2nd_Road_Class', 'Pedestrian_Crossing-Human_Control','Pedestrian_Crossing-Physical_Facilities',	'Light_Conditions',
 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'state', 'Road_Type', 'Speed_limit', 'Urban_or_Rural_Area' , 'Time_category', 
 'hour', 'minutes', 'year', 'month', ]


In [106]:
from sklearn.preprocessing import OneHotEncoder

one_enc = OneHotEncoder(sparse=False,handle_unknown='ignore')
x = one_enc.fit_transform(train_df[cat_columns])


In [108]:
train_df = pd.concat([train_df, pd.DataFrame(x)], axis = 1)
train_df.shape

(478741, 245)

In [107]:
x_test = one_enc.transform(test_df[cat_columns], )
test_df = pd.concat([test_df, pd.DataFrame(x_test)], axis = 1)
test_df.shape

(121259, 245)

In [111]:
one_enc.categories_

[array([1, 2, 3, 4, 5, 6, 7], dtype=int64),
 array([1, 3, 4, 5, 6], dtype=int64),
 array([-1,  3,  4,  5,  6], dtype=int64),
 array(['Control by other authorised person',
        'Control by school crossing patrol', 'None within 50 metres'],
       dtype=object),
 array(['Central refuge', 'Footbridge or subway',
        'No physical crossing within 50 meters',
        'Pedestrian phase at traffic signal junction', 'Zebra crossing',
        'non-junction pedestrian crossing'], dtype=object),
 array(['Darkeness: No street lighting',
        'Darkness: Street lighting unknown',
        'Darkness: Street lights present and lit',
        'Darkness: Street lights present but unlit',
        'Daylight: Street light present'], dtype=object),
 array(['Fine with high winds', 'Fine without high winds', 'Fog or mist',
        'Other', 'Raining with high winds', 'Raining without high winds',
        'Snowing with high winds', 'Snowing without high winds', 'Unknown'],
       dtype=object),
 array(['

In [112]:
train_df.to_csv('data_feather/train_ungrouped.csv')
test_df.to_csv('data_feather/test_ungrouped.csv')

In [113]:
train_df.isna().sum()

postcode                                                                               0
postcode_short                                                                         0
Accident_ID                                                                            0
Police_Force                                                                           0
Number_of_Vehicles                                                                     0
Number_of_Casualties                                                                   0
Date                                                                                   0
Day_of_Week                                                                            0
Time                                                                                  14
Local_Authority_(District)                                                             0
Local_Authority_(Highway)                                                              0
1st_Road_Class       